In [1]:
import requests

import pandas as pd
import numpy as np


In [2]:
def get_data_from_cbio_portal(query,):
    url = 'http://www.cbioportal.org/webservice.do'
    str_query = url + '?' + query
    return pd.read_table(str_query, encoding='iso-8859-1')

def modify_array_to_string(value):
    return str(value).strip().replace('[','').replace(']', '').replace(',', '+').replace('\'','').replace(' ', '')

In [3]:
def get_all_types_of_cancer():
    '''
    GET ALL TYPES OF CANCER
    RETURN PANDAS DATAFRAME:
        type_of_cancer_id: a unique text identifier used to identify the type of cancer. 
            For example, "gbm" identifies Glioblastoma multiforme.
        name: short name of the type of cancer.
    '''
    query = 'cmd=getTypesOfCancer'
    return get_data_from_cbio_portal(query=query)

def get_all_cancer_studies():
    '''
    GET ALL CANCER STUDIES
    RETURN PANDAS DATAFRAME:
        cancer_study_id: a unique ID that should be used to identify the cancer study in subsequent interface calls.
        name: short name of the cancer study.
        description: short description of the cancer study.
    '''
    query = 'cmd=getCancerStudies'
    return get_data_from_cbio_portal(query=query)

def get_all_genetic_profiles_for_specific_cancer_study(cancer_id):
    '''
    GET ALL GENETIC PROFILES FOR A SPECIFIC CANCER STUDY
    PARAMETERS:
        cancer_study_id=[cancer study ID] (required)
    RETURN PANDAS DATAFRAME:
        genetic_profile_id: a unique ID used to identify the genetic profile ID in subsequent interface calls. This is a human readable ID. For example, "gbm_mutations" identifies the TCGA GBM mutation genetic profile.
        genetic_profile_name: short profile name.
        genetic_profile_description: short profile description.
        cancer_study_id: cancer study ID tied to this genetic profile. Will match the input cancer_study_id.
        genetic_alteration_type: indicates the profile type. Will be one of:
            MUTATION
            MUTATION_EXTENDED
            COPY_NUMBER_ALTERATION
            MRNA_EXPRESSION
            METHYLATION
        show_profile_in_analysis_tab: a boolean flag used for internal purposes (you can safely ignore it).
    '''
    if type(cancer_id) is list:
        dfs = [get_data_from_cbio_portal(query='cmd=getGeneticProfiles&cancer_study_id=' + str(q)) for q in cancer_id]
        return pd.concat(dfs, ignore_index=True)
    
    query = 'cmd=getGeneticProfiles&cancer_study_id=' + str(cancer_id)
    return get_data_from_cbio_portal(query=query)

def get_all_case_list_for_specific_cancer_study(cancer_study_id):
    '''
    GET ALL CASE LISTS FOR A SPECIFIC CANCER STUDY
    PARAMETERS:
        cancer_study_id=[cancer study ID] (required)
    RETURN PANDAS DATAFRAME:
        case_list_id: a unique ID used to identify the case list ID in subsequent interface calls. This is a human readable ID. For example, "gbm_all" identifies all cases profiles in the TCGA GBM study.
        case_list_name: short name for the case list.
        case_list_description: short description of the case list.
        cancer_study_id: cancer study ID tied to this genetic profile. Will match the input cancer_study_id.
        case_ids: space delimited list of all case IDs that make up this case list.
    '''
    if type(cancer_id) is list:
        dfs = [get_data_from_cbio_portal(query='cmd=getCaseLists&cancer_study_id=' + str(q)) for q in cancer_id]
        return pd.concat(dfs, ignore_index=True)
    
    query = 'cmd=getCaseLists&cancer_study_id=' + str(cancer_study_id)
    return get_data_from_cbio_portal(query=query)


def get_clinical_data(case_set_id):
    '''
    GET CLINICAL DATA
    PARAMETERS:
        case_set_id= [case set ID] (required)
   RETURN PANDAS DATAFRAME:
        case_id: Unique Case Identifier.
        overall_survival_months: Overall survival, in months.
        overall_survival_status: Overall survival status, usually indicated as "LIVING" or "DECEASED".
        disease_free_survival_months: Disease free survival, in months.
        disease_free_survival_status: Disease free survival status, usually indicated as "DiseaseFree" or "Recurred/Progressed".
        age_at_diagnosis: Age at diagnosis.
    '''
    if type(case_set_id) is list:
        dfs = [get_data_from_cbio_portal(query='cmd=getClinicalData&case_set_id=' + str(q)) for q in case_set_id]
        return pd.concat(dfs, ignore_index=True)

    query = 'cmd=getClinicalData&case_set_id=' + str(case_set_id)
    return get_data_from_cbio_portal(query=query)

def get_extended_mutation_data(genetic_profile_id, case_set_id, gene_list):
    '''
    GET EXTENDED MUTATION DATA
    PARAMETERS:
        genetic_profile_id= [one or more mutation profile IDs] (required). Multiple genetic profile IDs must be separated by comma (,) characters, or URL encoded spaces, e.g. +
        case_set_id= [case set ID] (optional). If not provided, all cases that have data in the specified mutation profiles will be queried.
        gene_list= [one or more genes, specified as HUGO Gene Symbols or Entrez Gene IDs] (required). Multiple genes must be separated by comma (,) characters, or URL encoded spaces, e.g. +
    RETURN PANDAS DATAFRAME:
        entrez_gene_id: Entrez Gene ID.
        gene_symbol: HUGO Gene Symbol.
        case_id: Case ID.
        sequencing_center: Sequencer Center responsible for identifying this mutation. For example: broad.mit.edu.
        mutation_status: somatic or germline mutation status. all mutations returned will be of type somatic.
        mutation_type: mutation type, such as nonsense, missense, or frameshift_ins.
        validation_status: validation status. Usually valid, invalid, or unknown.
        amino_acid_change: amino acid change resulting from the mutation.
        functional_impact_score: predicted functional impact score, as predicted by: Mutation Assessor.
        xvar_link: Link to the Mutation Assessor web site.
        xvar_link_pdb: Link to the Protein Data Bank (PDB) View within Mutation Assessor web site.
        xvar_link_msa: Link the Multiple Sequence Alignment (MSA) view within the Mutation Assessor web site.
        chr: chromosome where mutation occurs.
        start_position: start position of mutation.
        end_position: end position of mutation.
        genetic_profile_id: mutation profile id.
    '''
    if type(genetic_profile_id) is list:
        genetic_profile_id = modify_array_to_string(genetic_profile_id)
        
    if type(gene_list) is list:
        gene_list = modify_array_to_string(gene_list)
        
    if type(case_set_id) is list:
        dfs = [get_data_from_cbio_portal(query='cmd=getMutationData&case_set_id=' + str(q) + '&genetic_profile_id=' + \
                                        genetic_profile_id + '&gene_list=' + gene_list) for q in case_set_id]
        return pd.concat(dfs, ignore_index=True)
    
    query='cmd=getMutationData&case_set_id=' + str(case_set_id) + '&genetic_profile_id=' + \
                                        genetic_profile_id + '&gene_list=' + gene_list
        
    return get_data_from_cbio_portal(query=query) 
    
    
def get_profile_data(case_set_id, genetic_profile_id, gene_list):
    '''
    GET PROFILE DATA
    PARAMETERS:
        cmd=getProfileData (required)
        case_set_id= [case set ID] (required)
        genetic_profile_id= [one or more genetic profile IDs] (required). Multiple genetic profile IDs must be separated by comma (,) characters, or URL encoded spaces, e.g. +
        gene_list= [one or more genes, specified as HUGO Gene Symbols or Entrez Gene IDs] (required). Multiple genes must be separated by comma (,) characters, or URL encoded spaces, e.g. +
    RETURN PANDAS DATAFRAME:
        When requesting one or multiple genes and a single genetic profile ID (see above), you will receive a tab-delimited matrix with the following columns:

        GENE_ID: Entrez Gene ID
        COMMON: HUGO Gene Symbol
        Columns 3 - N: Data for each case
        Response Format 2
        When requesting a single gene and multiple genetic profile IDs (see above), you will receive a tab-delimited matrix with the following columns:

        GENETIC_PROFILE_ID: The Genetic Profile ID.
        ALTERATION_TYPE: The Genetic Alteration Type, e.g. MUTATION, MUTATION_EXTENDED, COPY_NUMBER_ALTERATION, or MRNA_EXPRESSION.
        GENE_ID: Entrez Gene ID.
        COMMON: HUGO Gene Symbol.
        Columns 5 - N: Data for each case.
    '''
    if type(genetic_profile_id) is list:
        genetic_profile_id = modify_array_to_string(genetic_profile_id)
        
    if type(gene_list) is list:
        gene_list = modify_array_to_string(gene_list)
        
    if type(case_set_id) is list:
        dfs = [get_data_from_cbio_portal(query='cmd=getProfileData&case_set_id=' + str(q) + '&genetic_profile_id=' + \
                                        genetic_profile_id + '&gene_list=' + gene_list) for q in case_set_id]
        return pd.concat(dfs, ignore_index=True)
    
    query='cmd=getProfileData&case_set_id=' + str(case_set_id) + '&genetic_profile_id=' + \
                                        genetic_profile_id + '&gene_list=' + gene_list
        
    return get_data_from_cbio_portal(query=query)
 

In [4]:
get_clinical_data(case_set_id=['gbm_tcga_all', 'all_stjude_2016_sequenced'])

AGE   AGE_CLASS CANCER_TYPE     CANCER_TYPE_DETAILED  \
0    47.00         NaN      Glioma  Glioblastoma Multiforme   
1    60.00         NaN      Glioma  Glioblastoma Multiforme   
2    63.00         NaN      Glioma  Glioblastoma Multiforme   
3    52.00         NaN      Glioma  Glioblastoma Multiforme   
4    59.00         NaN      Glioma  Glioblastoma Multiforme   
5    21.00         NaN      Glioma  Glioblastoma Multiforme   
6    85.00         NaN      Glioma  Glioblastoma Multiforme   
7    26.00         NaN      Glioma  Glioblastoma Multiforme   
8    71.00         NaN      Glioma  Glioblastoma Multiforme   
9    45.00         NaN      Glioma  Glioblastoma Multiforme   
10   46.00         NaN      Glioma  Glioblastoma Multiforme   
11   46.00         NaN      Glioma  Glioblastoma Multiforme   
12   54.00         NaN      Glioma  Glioblastoma Multiforme   
13   24.00         NaN      Glioma  Glioblastoma Multiforme   
14   75.00         NaN      Glioma  Glioblastoma Multiforme   
15   67.00         NaN      Glioma  Glioblastoma Multiforme   
16   50.00         NaN      Glioma  Glioblastoma Multiforme   
17   69.00         NaN      Glioma  Glioblastoma Multiforme   
18   48.00         NaN      Glioma  Glioblastoma Multiforme   
19   56.00         NaN      Glioma  Glioblastoma Multiforme   
20   63.00         NaN      Glioma  Glioblastoma Multiforme   
21   17.00         NaN      Glioma  Glioblastoma Multiforme   
22   57.00         NaN      Glioma  Glioblastoma Multiforme   
23   45.00         NaN      Glioma  Glioblastoma Multiforme   
24   67.00         NaN      Glioma  Glioblastoma Multiforme   
25   51.00         NaN      Glioma  Glioblastoma Multiforme   
26   53.00         NaN      Glioma  Glioblastoma Multiforme   
27   78.00         NaN      Glioma  Glioblastoma Multiforme   
28   74.00         NaN      Glioma  Glioblastoma Multiforme   
29   57.00         NaN      Glioma  Glioblastoma Multiforme   
..     ...         ...         ...                      ...   
647   1.29   Childhood    Leukemia  Acute Lymphoid Leukemia   
648  13.17   Childhood    Leukemia  Acute Lymphoid Leukemia   
649  17.50  Adolescent    Leukemia  Acute Lymphoid Leukemia   
650  19.68  Adolescent    Leukemia  Acute Lymphoid Leukemia   
651  16.67  Adolescent    Leukemia  Acute Lymphoid Leukemia   
652  15.73   Childhood    Leukemia  Acute Lymphoid Leukemia   
653   7.98   Childhood    Leukemia  Acute Lymphoid Leukemia   
654  13.79   Childhood    Leukemia  Acute Lymphoid Leukemia   
655  10.86   Childhood    Leukemia  Acute Lymphoid Leukemia   
656  10.78   Childhood    Leukemia  Acute Lymphoid Leukemia   
657   6.98   Childhood    Leukemia  Acute Lymphoid Leukemia   
658   4.95   Childhood    Leukemia  Acute Lymphoid Leukemia   
659  11.50   Childhood    Leukemia  Acute Lymphoid Leukemia   
660  10.92   Childhood    Leukemia  Acute Lymphoid Leukemia   
661  15.69   Childhood    Leukemia  Acute Lymphoid Leukemia   
662  12.05   Childhood    Leukemia  Acute Lymphoid Leukemia   
663  15.62   Childhood    Leukemia  Acute Lymphoid Leukemia   
664  10.78   Childhood    Leukemia  Acute Lymphoid Leukemia   
665  10.92   Childhood    Leukemia  Acute Lymphoid Leukemia   
666  13.42   Childhood    Leukemia  Acute Lymphoid Leukemia   
667   6.00   Childhood    Leukemia  Acute Lymphoid Leukemia   
668  18.02  Adolescent    Leukemia  Acute Lymphoid Leukemia   
669   3.34   Childhood    Leukemia  Acute Lymphoid Leukemia   
670   5.45   Childhood    Leukemia  Acute Lymphoid Leukemia   
671   6.98   Childhood    Leukemia  Acute Lymphoid Leukemia   
672   9.36   Childhood    Leukemia  Acute Lymphoid Leukemia   
673   4.13   Childhood    Leukemia  Acute Lymphoid Leukemia   
674  15.53   Childhood    Leukemia  Acute Lymphoid Leukemia   
675   7.48   Childhood    Leukemia  Acute Lymphoid Leukemia   
676   9.99   Childhood    Leukemia  Acute Lymphoid Leukemia   

                 CASE_ID                                       CYTOGENETICS  \
0        TCGA-32-4213

In [5]:
get_all_types_of_cancer()

type_of_cancer_id                                               name
0                  aa                             Aggressive Angiomyxoma
1               aastr                             Anaplastic Astrocytoma
2                 abc                              Activated B-cell Type
3                 abl                          Acute Basophilic Leukemia
4                 aca                             Adrenocortical Adenoma
5                acbc                       Adenoid Cystic Breast Cancer
6                 acc                           Adrenocortical Carcinoma
7                accc                              Acinic Cell Carcinoma
8                acml       Atypical Chronic Myeloid Leukemia, BCR-ABL1-
9                 acn                         Acinar Cell Carcinoma, NOS
10               acpg           Craniopharyngioma, Adamantinomatous Type
11               acpp                  Atypical Choroid Plexus Papilloma
12               acrm                                     Acral Melanoma
13               acyc                           Adenoid Cystic Carcinoma
14               adma                                       Adamantinoma
15              adnos                                Adenocarcinoma, NOS
16               adpa        Aggressive Digital Papillary Adenocarcinoma
17      adrenal_gland                                      Adrenal Gland
18               aeca   Sweat Gland Carcinoma/Apocrine Eccrine Carcinoma
19                afx                             Atypical Fibroxanthoma
20                aga                          Anal Gland Adenocarcinoma
21               agng                           Anaplastic Ganglioglioma
22               ahcd                          Alpha Heavy-Chain Disease
23                ais                             Adenocarcinoma In Situ
24               aitl                 Angioimmunoblastic T-Cell Lymphoma
25               alal               Acute Leukemias of Ambiguous Lineage
26               alcl                     Anaplastic Large Cell Lymphoma
27           alclalkn        Anaplastic Large-Cell Lymphoma ALK Negative
28           alclalkp        Anaplastic Large-Cell Lymphoma ALK Positive
29            alklbcl                 ALK Positive Large B-Cell Lymphoma
..                ...                                                ...
810               usc  Uterine Serous Carcinoma/Uterine Papillary Ser...
811              uscc                   Urethral Squamous Cell Carcinoma
812              usmt                        Uterine Smooth Muscle Tumor
813             ustad            Undifferentiated Stomach Adenocarcinoma
814            ustump  Uterine Smooth Muscle Tumor of Uncertain Malig...
815            uterus                                             Uterus
816              utuc                   Upper Tract Urothelial Carcinoma
817               uuc                 Uterine Undifferentiated Carcinoma
818               uus                   Undifferentiated Uterine Sarcoma
819                va                             Vaginal Adenocarcinoma
820              vdys                                       Dysgerminoma
821              vgce        Villoglandular Adenocarcinoma of the Cervix
822              vgct                       Germ Cell Tumor of the Vulva
823              vimt                                  Immature Teratoma
824               vma        Mucinous Adenocarcinoma of the Vulva/Vagina
825             vmgct                              Mixed Germ Cell Tumor
826               vmm               Mucosal Melanoma of the Vulva/Vagina
827               vmt                                    Mature Teratoma
828              voec                                Embryonal Carcinoma
829              vpdc            Poorly Differentiated Vaginal Carcinoma
830               vpe                                       Polyembryoma
831             vpscc           Verrucous Penile Squamous Cell Carcinoma
832               vsc        Squamous Cell Carcinoma of the Vu

In [6]:
get_all_cancer_studies()

cancer_study_id  \
0                   paac_jhu_2014   
1                 all_stjude_2016   
2                   laml_tcga_pub   
3    laml_tcga_pan_can_atlas_2018   
4                       laml_tcga   
5                   acyc_fmi_2014   
6                   acyc_mda_2015   
7                 acyc_mskcc_2013   
8                acyc_sanger_2013   
9                 acbc_mskcc_2015   
10    acc_tcga_pan_can_atlas_2018   
11                       acc_tcga   
12                 ampca_bcm_2016   
13          blca_mskcc_solit_2014   
14          blca_mskcc_solit_2012   
15             blca_tcga_pub_2017   
16   blca_plasmacytoid_mskcc_2016   
17                       blca_bgi   
18        blca_dfarber_mskcc_2014   
19                  blca_tcga_pub   
20   blca_tcga_pan_can_atlas_2018   
21                      blca_tcga   
22    lgg_tcga_pan_can_atlas_2018   
23                       lgg_tcga   
24                  brca_metabric   
25                     brca_bccrc   
26                     brca_broad   
27                    brca_sanger   
28              brca_tcga_pub2015   
29                  brca_tcga_pub   
..                            ...   
194  stad_tcga_pan_can_atlas_2018   
195                     stad_tcga   
196                   stad_utokyo   
197                stad_uhongkong   
198                 stes_tcga_pub   
199               urcc_mskcc_2016   
200                  crc_msk_2018   
201               utuc_mskcc_2013   
202                     tgct_tcga   
203  tgct_tcga_pan_can_atlas_2018   
204     angs_project_painter_2018   
205    brca_mbcproject_wagle_2017   
206                  tet_nci_2014   
207  thym_tcga_pan_can_atlas_2018   
208                     thym_tcga   
209  thca_tcga_pan_can_atlas_2018   
210                     thca_tcga   
211                  ucs_jhu_2014   
212   ucs_tcga_pan_can_atlas_2018   
213                      ucs_tcga   
214                 ucec_tcga_pub   
215  ucec_tcga_pan_can_atlas_2018   
216                     ucec_tcga   
217   uvm_tcga_pan_can_atlas_2018   
218                      uvm_tcga   
219             panet_arcnet_2017   
220                skcm_ucla_2016   
221            cscc_hgsc_bcm_2014   
222     past_dkfz_heidelberg_2013   
223         hcc_msk_venturaa_2018   

                                                  name  \
0    Acinar Cell Carcinoma of the Pancreas (Johns H...   
1    Acute Lymphoblastic Leukemia (St Jude, Nat Gen...   
2             Acute Myeloid Leukemia (TCGA, NEJM 2013)   
3       Acute Myeloid Leukemia (TCGA, PanCancer Atlas)   
4           Acute Myeloid Leukemia (TCGA, Provisional)   
5    Adenoid Cystic Carcinoma (FMI, Am J Surg Pathl...   
6    Adenoid Cystic Carcinoma (MDA, Clin Cancer Res...   
7     Adenoid Cystic Carcinoma (MSKCC, Nat Genet 2013)   
8      Adenoid Cystic Carcinoma (Sanger/MDA, JCI 2013)   
9    Adenoid Cystic Carcinoma of the Breast (MSKCC,...   
10    Adrenocortical Carcinoma (TCGA, PanCancer Atlas)   
11        Adrenocortical Carcinoma (TCGA, Provisional)   
12   Ampullary Carcinoma (Baylor College of Medicin...   
13               Bladder Cancer (MSKCC, Eur Urol 2014)   
14                    Bladder Cancer (MSKCC, JCO 2013)   
15                    Bladder Cancer (TCGA, Cell 2017)   
16   Bladder Cancer, Plasmacytoid Variant (MSKCC, N...   
17   Bladder Urothelial Carcinoma (BGI, Nat Genet 2...   
18   Bladder Urothelial Carcinoma (Dana Farber & MS...   
19    Bladder Urothelial Carcinoma (TCGA, Nature 2014)   
20   Bladder Urothelial Carcinoma (TCGA, PanCancer ...   
21    Bladder Urothelial Carcinoma (TCGA, Provisional)   
22    Brain Lower Grade Glioma (TCGA, PanCancer Atlas)   
23        Brain Lower Grade Glioma (TCGA, Provisional)   
24   Breast Cancer (METABRIC, Nature 2012 & Nat Com...   
25   Breast Invasive Carcinoma (British Columbia, N...   
26      Breast Invasive Carcinoma (Broad, Nature 2012)   
27     Breast Invasive Carcinoma (Sanger, Nature 2012)   
28         Breast Invasive Carcinoma (TCG

In [7]:
cancer_id=['all_stjude_2016', 'paac_jhu_2014']
case_id=['SJBALL020595_D1', 'SJERG005_D']

In [8]:
get_all_case_list_for_specific_cancer_study(cancer_id)

case_list_id    case_list_name  \
0        all_stjude_2016_all        All Tumors   
1  all_stjude_2016_sequenced  Sequenced Tumors   
2          paac_jhu_2014_all        All Tumors   
3    paac_jhu_2014_sequenced  Sequenced Tumors   

                           case_list_description  cancer_study_id  \
0                 All tumor samples (73 samples)              920   
1  All (Next-Gen) sequenced samples (73 samples)              920   
2                 All tumor samples (23 samples)              815   
3  All (Next-Gen) sequenced samples (23 samples)              815   

                                            case_ids  
0  SJBALL020595_D1 SJERG005_D SJERG023_D_WES SJER...  
1  SJBALL020595_D1 SJERG005_D SJERG023_D_WES SJER...  
2  ACINAR01 ACINAR02 ACINAR03 ACINAR04 ACINAR05 A...  
3  ACINAR01 ACINAR02 ACINAR03 ACINAR04 ACINAR05 A...

In [9]:
get_all_genetic_profiles_for_specific_cancer_study(cancer_id)

genetic_profile_id genetic_profile_name  \
0  all_stjude_2016_mutations            Mutations   
1    paac_jhu_2014_mutations            Mutations   

                         genetic_profile_description  cancer_study_id  \
0                        Mutation data from WGS/WES.              920   
1  Mutation data from whole exome sequencing of 2...              815   

  genetic_alteration_type  show_profile_in_analysis_tab  
0       MUTATION_EXTENDED                          True  
1       MUTATION_EXTENDED                          True